In [1]:
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)


In [2]:
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Input, MaxPooling2D, UpSampling2D
from tensorflow.keras.layers import BatchNormalization, Concatenate, Activation, Dropout
from keras.optimizers import Adam
from keras import initializers
from tensorflow.keras.models import Model
from keras import backend as K
from dataGenerator import gen_train_data, test_data
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
from datetime import datetime

from sklearn.model_selection import KFold


Using TensorFlow backend.


In [ ]:
#path_weight = 'model_weight.h5'

In [3]:
# accuracy and loss
def iu_acc(y_true, y_pred):
    # __author__ = Vladimir Iglovikov
    smooth = 1e-12
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    intersection = K.sum(y_true * y_pred_pos, axis=[1, 2, 3], keepdims=False)
    sum_ = K.sum(y_true + y_pred_pos, axis=[1, 2, 3], keepdims=False)
    jac = (intersection) / (sum_ - intersection + smooth)
    return K.mean(jac)

def dice_acc(y_true, y_pred):
    smooth = 1e-12
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    intersection = K.sum(y_true * y_pred_pos, axis=[1, 2, 3], keepdims=False)
    sum_ = K.sum(y_true + y_pred_pos, axis=[1, 2, 3], keepdims=False)
    jac = (2*intersection + smooth) / (sum_ + smooth)
    return K.mean(jac)

# loss function
def my_loss(y_true,y_pred):
    bc = K.binary_crossentropy(y_pred, y_true)
    false_bc = (1.0 - y_true) * bc
    true_bc = y_true * bc
    mloss = false_bc + 10.0 * true_bc
    return mloss

smooth = 1.
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

# dice_coef may have better performance
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)


def ConvActive(np_ker, size, inputs, 
                        initial = initializers.glorot_normal(seed=None)):
    
    return Conv2D(np_ker, size, activation='relu',padding = 'same', kernel_initializer = initial)(inputs)

def ConvBatchnormActive(np_ker, size, inputs, 
                        initial = initializers.glorot_normal(seed=None)):
    
    return Activation('relu')(BatchNormalization()(Conv2D(np_ker, size,padding = 'same', kernel_initializer = initial)(inputs)))

def UpConvActiveContact(np_ker, size, inputs, contact,
                  initial = initializers.glorot_normal(seed=None)):
    up = UpSampling2D(size = (2,2))(inputs)
    upconv = Conv2D(np_ker, size, activation = 'relu', padding = 'same', kernel_initializer = initial)(up) 
    return Concatenate(axis=3)([upconv, contact])

def get_unet(pretrained_weights = None):
    
    inputs = Input((512, 512, 3))

    conv1 = ConvBatchnormActive(16, 3, inputs)
    conv1 = ConvBatchnormActive(16, 3, conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    pool1 = Dropout(0.25)(pool1)
    
    conv2 = ConvBatchnormActive(32, 3, pool1)
    conv2 = ConvBatchnormActive(32, 3, conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    pool2 = Dropout(0.5)(pool2)
    
    conv3 = ConvBatchnormActive(64, 3, pool2)
    conv3 = ConvBatchnormActive(64, 3, conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    pool3 = Dropout(0.5)(pool3)
    
    conv4 = ConvBatchnormActive(128, 3, pool3)
    conv4 = ConvBatchnormActive(128, 3, conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    pool4 = Dropout(0.5)(pool4)
    
    conv5 = ConvBatchnormActive(256, 3, pool4)
    conv5 = ConvBatchnormActive(256, 3, conv5)
    
    up6 = UpConvActiveContact(128, 2, conv5, conv4)
    up6 = Dropout(0.5)(up6)
    conv6 = ConvBatchnormActive(128, 3, up6)
    conv6 = ConvBatchnormActive(128, 3, conv6)
    
    up7 = UpConvActiveContact(64, 2, conv6, conv3)
    up7 = Dropout(0.5)(up7)
    conv7 = ConvBatchnormActive(64, 3, up7)
    conv7 = ConvBatchnormActive(64, 3, conv7)
    
    up8 = UpConvActiveContact(32, 2, conv7, conv2)
    up8 = Dropout(0.5)(up8)
    conv8 = ConvBatchnormActive(32, 3, up8)
    conv8 = ConvBatchnormActive(32, 3, conv8)
    
    up9 = UpConvActiveContact(16, 2, conv8, conv1)
    up9 = Dropout(0.5)(up9)
    conv9 = ConvBatchnormActive(16, 3, up9)
    conv9 = ConvBatchnormActive(16, 3, conv9)
    
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(inputs = inputs, outputs = conv10)
    
    model.compile(optimizer = 'Adam', loss = dice_coef_loss, metrics = [iu_acc, dice_acc, 'accuracy'])

    if(pretrained_weights):
        model.load_weights(pretrained_weights)

    return model



In [4]:
model = get_unet()
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 512, 512, 16) 448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 512, 512, 16) 64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 512, 512, 16) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [ ]:
#%% training from scratch

model_checkpoint = ModelCheckpoint('D:/Anil/saved_model/gray_mold/k-fold-cv/Unet_kf1_50epoch_10000_it.hdf5', monitor = 'loss', verbose = 1, save_best_only =True)
callback_checkpoint = [model_checkpoint]

path_train = 'D:/Anil/Dataset/gray_mold/k-fold/kf1/train/'
path_test = 'D:/Anil/Dataset/gray_mold/k-fold/kf1/test/'

# model = get_unet()
train_data = gen_train_data(path_train)
test_data = test_data(path_test)
start_time = datetime.now()
print("Start time: {}".format(start_time))
history = model.fit(train_data, steps_per_epoch=10000, epochs=50, verbose=1, 
                    shuffle=True, validation_data=test_data, callbacks = callback_checkpoint)

end_time = datetime.now()
total_time = end_time - start_time
print("Total time taken: {}".format(total_time))


Start time: 2021-07-29 08:23:42.315459
Found 90 images belonging to 1 classes.
Found 90 images belonging to 1 classes.
Epoch 1/50
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decora

In [ ]:
import pandas as pd
history_dict = pd.DataFrame(history.history) 
hist_csv_file = "D:/Anil/saved_model/gray_mold/k-fold-cv/kf1/kf1_model_history_50epochs_10000_it.csv"
with open(hist_csv_file, mode='w', newline = '') as f:
    history_dict.to_csv(f)
    